# Exercise 5

In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

FS = 15 # Fontsize of caption

## 5.1 Global Histogram Equalization

In [3]:
# Read the image test
image = cv2.imread('moon.jpg')

# Calculate histogram before equalization
hist_before, bins_before = np.histogram(image.flatten(), bins = 256, range = [0, 256])

# Apply histogram equalization
equalized_image = cv2.equalizeHist(image)

# Calculate histogram after equalization
hist_after, bins_after = np.histogram(equalized_image.flatten(), bins = 256, range = [0, 256])

# Plot original image and its histogram
plt.figure(figsize=(12, 6))

plt.subplot(2, 2, 1)
plt.imshow(image)
plt.title('Original Image', fontsize = FS)
plt.axis('off')

plt.subplot(2, 2, 2)
plt.bar(bins_before[:-1], hist_before, width=1, color='gray', edgecolor='black')
plt.title('Histogram Before Equalization', fontsize = FS)
plt.xlabel('Gray Level', fontsize = FS)
plt.ylabel('Frequency', fontsize = FS)
plt.grid(True)

# Plot equalized image and its histogram
plt.subplot(2, 2, 3)
plt.imshow(equalized_image)
plt.title('Equalized Image', fontsize = FS)
plt.axis('off')

plt.subplot(2, 2, 4)
plt.bar(bins_after[:-1], hist_after, width=1, color='gray', edgecolor='black')
plt.title('Histogram After Equalization', fontsize = FS)
plt.xlabel('Gray Level', fontsize = FS)
plt.ylabel('Frequency', fontsize = FS)
plt.grid(True)

plt.tight_layout()
plt.show()

error: OpenCV(4.10.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/histogram.cpp:3440: error: (-215:Assertion failed) _src.type() == CV_8UC1 in function 'equalizeHist'
